In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
import torch

# Load Finbert model and tokenizer
model = BertForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone", num_labels=3)
tokenizer = BertTokenizer.from_pretrained("yiyanghkust/finbert-tone")

# Function: preprocess text and convert to tensor
def preprocess_text(text, max_length=102):
    inputs = tokenizer(text, padding=True, truncation=True, max_length = max_length, return_tensors="pt")
    return inputs

# Function: get sentiment label
def get_sentiment_label(logits):
    probabilities = torch.softmax(logits, dim=1)
    sentiment = torch.argmax(probabilities, dim=1).item()
    return sentiment

# Load data from datafile for analysis
data = pd.read_csv("/Users/willgovier/Documents/Interdisciplinary group project/GROUP PROJECT/Data repo/IGP-Data/Prepared Data/Stock News/MSFT/MSFT_stock_news_2024-04-15_122011.csv")

# Extract headlines only for sentiment analysis in this case 
headlines = data["headline"]

# Empty list to store sentiment labels
sentiments = []

# Set device for model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Process each headline
for headline in headlines:
    # Preprocess the text
    inputs = preprocess_text(headline)
    inputs = {key: inputs[key].to(device) for key in inputs}

    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    # Get sentiment label
    sentiment = get_sentiment_label(outputs.logits)
    sentiments.append(sentiment)

# Add sentiment labels to the DataFrame
data["sentiment"] = sentiments

# Map sentiment labels to their respective categories (0: Negative, 1: Neutral, 2: Positive)
sentiment_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
data["sentiment"] = data["sentiment"].map(sentiment_map)

# Print the DataFrame with sentiment labels
print(data)

                                             headline  \
0        2 Powerhouse ETFs to Buy With $500 Right Now   
1   Riding the AI Wave: Why Microsoft Stock Remain...   
2   3 Top Artificial Intelligence (AI) ETFs That A...   
3   OpenAI Taps Ex-Amazon Executive to Head Enterp...   
4   UPDATE 2-OpenAI bids for Japan business as it ...   
..                                                ...   
95  UPDATE 1-Japan's Kishida expects more US chip ...   
96  Nvidia, Google Expand Partnership With Nvidia ...   
97  Intel's latest AI chip is a direct shot at Nvi...   
98  Microsoft (MSFT) Plans to Establish New AI Hub...   
99  Google Announces Its First Arm-Based Custom Ch...   

                                                 link           date sentiment  
0   https://finance.yahoo.com/m/d0ef1bec-6716-3000...  Today 07:00AM  Negative  
1   https://investorplace.com/market360/2024/04/ri...        06:00AM   Neutral  
2   https://finance.yahoo.com/m/1fd17b8b-d7f1-3a84...        04:43AM  Ne